## Context

ABC Supermarket is planning for the year-end sale - they want to launch a new offer i.e. gold membership for only \\$499 that is of \\$999 on normal days (that gives 20% discount on all purchases) only for existing customers, for that they need to do a campaign through phone calls - the best way to reduce the cost of the campaign is to make a predictive model to classify customers who might purchase the offer, using the data they gathered during last year's campaign.

We will build a model for classifying whether customers will reply with a positive
response or not.

## Column Definitions
- Response (target) - 1 if customer accepted the offer in the last campaign, 0
otherwise
- Complain - 1 if a customer complained in the last 2 years
- DtCustomer - date of customer’s enrolment with the company
- Education - customer’s level of education
- Marital - customer’s marital status
- Kidhome - number of small children in customer’s household
- Teenhome - number of teenagers in customer’s household
- Income - customer’s yearly household income
- MntFishProducts - the amount spent on fish products in the last 2 years
- MntMeatProducts - the amount spent on meat products in the last 2 years
- MntFruits - the amount spent on fruits products in the last 2 years
- MntSweetProducts - amount spent on sweet products in the last 2 years
- MntWines - the amount spent on wine products in the last 2 years
- MntGoldProds - the amount spent on gold products in the last 2 years
- NumDealsPurchases - number of purchases made with discount
- NumCatalogPurchases - number of purchases made using catalog
- NumStorePurchases - number of purchases made directly in stores
- NumWebPurchases - number of purchases made through the company’s web site
- NumWebVisitsMonth - number of visits to company’s web site in the last month
- Recency - number of days since the last purchase
- ID - unique customer-id
- Year_Birth - customer's year of birth

## Import Libraries

In [1]:
# Import essential libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

## Loading the Dataset

In [2]:
df = pd.read_excel("marketing_data.xlsx")

## Initial Data Profiling

In [3]:
df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,Complain
0,1826,1970,Graduation,Divorced,84835.0,0,0,6/16/14,0,189,...,111,189,218,1,4,4,6,1,1,0
1,1,1961,Graduation,Single,57091.0,0,0,6/15/14,0,464,...,7,0,37,1,7,3,7,5,1,0
2,10476,1958,Graduation,Married,67267.0,0,1,5/13/14,0,134,...,15,2,30,1,3,2,5,2,0,0
3,1386,1967,Graduation,Together,32474.0,1,1,2014-11-05 00:00:00,0,10,...,0,0,0,1,1,0,2,7,0,0
4,5371,1989,Graduation,Single,21474.0,1,0,2014-08-04 00:00:00,0,6,...,11,0,34,2,3,1,2,7,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10142,1976,PhD,Divorced,66476.0,0,1,2013-07-03 00:00:00,99,372,...,47,48,78,2,5,2,11,4,0,0
2236,5263,1977,2n Cycle,Married,31056.0,1,0,1/22/13,99,5,...,3,8,16,1,1,0,3,8,0,0
2237,22,1976,Graduation,Divorced,46310.0,1,0,2012-03-12 00:00:00,99,185,...,15,5,14,2,6,1,5,8,0,0
2238,528,1978,Graduation,Married,65819.0,0,0,11/29/12,99,267,...,149,165,63,1,5,4,10,3,0,0


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,2240.0,5592.159821,3246.662198,0.0,2828.25,5458.5,8427.75,11191.0
Year_Birth,2240.0,1968.805804,11.984069,1893.0,1959.00,1970.0,1977.00,1996.0
Income,2216.0,52247.251354,25173.076661,1730.0,35303.00,51381.5,68522.00,666666.0
Kidhome,2240.0,0.444196,0.538398,0.0,0.00,0.0,1.00,2.0
Teenhome,2240.0,0.506250,0.544538,0.0,0.00,0.0,1.00,2.0
Recency,2240.0,49.109375,28.962453,0.0,24.00,49.0,74.00,99.0
MntWines,2240.0,303.935714,336.597393,0.0,23.75,173.5,504.25,1493.0
MntFruits,2240.0,26.302232,39.773434,0.0,1.00,8.0,33.00,199.0
MntMeatProducts,2240.0,166.950000,225.715373,0.0,16.00,67.0,232.00,1725.0
MntFishProducts,2240.0,37.525446,54.628979,0.0,3.00,12.0,50.00,259.0


In [5]:
df.isnull().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
Response                0
Complain                0
dtype: int64

## Taking Care of Missing Data

In [6]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy = 'mean')
df['Income'] = imputer.fit_transform(df['Income'].to_numpy().reshape(-1,1))

In [7]:
df.isnull().sum()

ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
Response               0
Complain               0
dtype: int64

## Feature Profiling

In [8]:
#pd.set_option("display.max_rows", None)
df['Recency'].value_counts()

Recency
56    37
54    32
30    32
46    31
49    30
      ..
5     15
59    14
22    13
7     12
44    11
Name: count, Length: 100, dtype: int64

In [9]:
df['NumWebPurchases'].value_counts()

NumWebPurchases
2     373
1     354
3     336
4     280
5     220
6     205
7     155
8     102
9      75
0      49
11     44
10     43
27      2
25      1
23      1
Name: count, dtype: int64

In [10]:
df['NumWebVisitsMonth'].value_counts()

NumWebVisitsMonth
7     393
8     342
6     340
5     281
4     218
3     205
2     202
1     153
9      83
0      11
10      3
20      3
14      2
19      2
17      1
13      1
Name: count, dtype: int64

In [11]:
df['MntGoldProds'].value_counts()

MntGoldProds
1      73
4      70
3      69
5      63
12     63
       ..
157     1
137     1
247     1
140     1
123     1
Name: count, Length: 213, dtype: int64

In [12]:
df['NumDealsPurchases'].value_counts()

NumDealsPurchases
1     970
2     497
3     297
4     189
5      94
6      61
0      46
7      40
8      14
9       8
15      7
11      5
10      5
12      4
13      3
Name: count, dtype: int64

In [13]:
df['MntWines'].value_counts()

MntWines
2       42
5       40
6       37
1       37
4       33
        ..
1302     1
109      1
545      1
863      1
169      1
Name: count, Length: 776, dtype: int64

In [14]:
df['Education'].value_counts()

Education
Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: count, dtype: int64

## Converting Year of Birth to Age

In [15]:
import pandas as pd
from datetime import datetime

# Assuming your DataFrame is named 'df'
# Convert 'Year_Birth' to datetime format
df['Year_Birth'] = pd.to_datetime(df['Year_Birth'], format='%Y')

# Calculate age by subtracting birth year from the current year
current_year = datetime.now().year
df['Age'] = current_year - df['Year_Birth'].dt.year

# Drop the original 'Year_Birth' column if needed
df = df.drop('Year_Birth', axis=1)

In [16]:
df

,ID,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,Complain,Age
0,1826,Graduation,Divorced,84835.0,0,0,6/16/14,0,189,104,...,189,218,1,4,4,6,1,1,0,54
1,1,Graduation,Single,57091.0,0,0,6/15/14,0,464,5,...,0,37,1,7,3,7,5,1,0,63
2,10476,Graduation,Married,67267.0,0,1,5/13/14,0,134,11,...,2,30,1,3,2,5,2,0,0,66
3,1386,Graduation,Together,32474.0,1,1,2014-11-05 00:00:00,0,10,0,...,0,0,1,1,0,2,7,0,0,57
4,5371,Graduation,Single,21474.0,1,0,2014-08-04 00:00:00,0,6,16,...,0,34,2,3,1,2,7,1,0,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10142,PhD,Divorced,66476.0,0,1,2013-07-03 00:00:00,99,372,18,...,48,78,2,5,2,11,4,0,0,48
2236,5263,2n Cycle,Married,31056.0,1,0,1/22/13,99,5,10,...,8,16,1,1,0,3,8,0,0,47
2237,22,Graduation,Divorced,46310.0,1,0,2012-03-12 00:00:00,99,185,2,...,5,14,2,6,1,5,8,0,0,48
2238,528,Graduation,Married,65819.0,0,0,11/29/12,99,267,38,...,165,63,1,5,4,10,3,0,0,46


## Removing Outliers

In [17]:
df.shape

(2240, 22)

## Encoding Categorical Data

In [18]:
from sklearn.preprocessing import OneHotEncoder
#Conversion of Categorical Variable to One-Hot Encoding
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output= False).set_output(transform = 'pandas')
df_transformed = ohe.fit_transform(df[['Education','Marital_Status']])
df = pd.concat([df,df_transformed], axis=1).drop(columns = ['Education','Marital_Status'])

In [19]:
df = df.drop(columns = ['Kidhome', 'Teenhome', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts','Dt_Customer', 'NumCatalogPurchases', 'NumStorePurchases'])

In [25]:
pd.set_option('display.max_columns', None)
df

,ID,Income,Recency,MntWines,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumWebVisitsMonth,Response,Complain,Age,Education_2n Cycle,Education_Basic,Education_Graduation,Education_Master,Education_PhD,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
0,1826,84835.0,0,189,218,1,4,1,1,0,54,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,57091.0,0,464,37,1,7,5,1,0,63,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10476,67267.0,0,134,30,1,3,2,0,0,66,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1386,32474.0,0,10,0,1,1,7,0,0,57,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5371,21474.0,0,6,34,2,3,7,1,0,35,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10142,66476.0,99,372,78,2,5,4,0,0,48,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2236,5263,31056.0,99,5,16,1,1,8,0,0,47,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2237,22,46310.0,99,185,14,2,6,8,0,0,48,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2238,528,65819.0,99,267,63,1,5,3,0,0,46,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## Splitting Test and Train sets

In [26]:
X = df.drop('Response', axis= 1)
y = df['Response']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42)

In [27]:
#Using MinMaxScaler because the distribution is not normal distribution
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Baseline Modelling

In [31]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
preds = clf.predict(X_test)

In [33]:
acc = accuracy_score(y_test, preds)
prec = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
auc = roc_auc_score(y_test, preds)

In [34]:
print("Accuracy: %.4f" % acc)
print("Precision: %.4f" % prec)
print("Recall: %.4f" % rec)
print("F1: %.4f" % f1)
print("AUC: %.4f" % auc)

Accuracy: 0.8438
Precision: 0.5714
Recall: 0.1111
F1: 0.1860
AUC: 0.5476
